# CL

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
%matplotlib inline
from pandas import read_excel
df_soc = pd.read_excel('soc.xlsx')#Social-demographical data (client ID, age, gender, tenure in a bank)
df_product = pd.read_excel('products.xlsx')#Products and actual balance (current account, saving account, mutual funds, overdraft, credit card, consumer loan)
df_inoutflow = pd.read_excel('inoutflow.xlsx')#Inflow/outflow on C/A, aggregated card turnover (monthly average over past 3 months)
df_sales = pd.read_excel('sales.xlsx')#Clients actual sales and revenues. Data is available for 60 % of the clients.

# TABLE SALE_CL

In [92]:
# table cl
df_train_cl = pd.merge(df_sales, df_inoutflow, on='Client', how='left')
df_train_cl = df_train_cl.drop(['Sale_MF','Sale_CC','Revenue_MF','Revenue_CC','Revenue_CL'],axis=1)
df_train_whole_cl = pd.merge(df_train_cl, df_soc, on="Client", how= "left")
df_train_whole_cl['Sex']=df_train_whole_cl['Sex'].replace({'M':'0','F':'1'})
df_train_whole_cl=df_train_whole_cl.dropna()
df_train_whole_cl

,Client,Sale_CL,VolumeCred,VolumeCred_CA,TransactionsCred,TransactionsCred_CA,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sex,Age,Tenure
0,1094,0,0.011786,0.011786,1.0,1.0,1366.313929,1366.313929,53.571429,351.950357,891.327857,31.0,31.0,1.0,21.0,4.0,0,25,8
1,1484,1,1355.088571,1355.088571,5.0,5.0,768.625000,768.625000,107.142857,302.446429,357.785714,23.0,23.0,1.0,16.0,5.0,0,39,0
2,20,0,452.617500,451.550000,4.0,3.0,221.428571,221.428571,107.142857,10.714286,103.571429,7.0,7.0,3.0,1.0,3.0,0,27,181
3,1193,0,1442.837857,1151.652857,23.0,21.0,1661.201071,1207.906429,507.142857,331.150000,75.178571,55.0,36.0,6.0,21.0,2.0,0,48,16
4,142,0,714.285714,714.285714,1.0,1.0,728.107143,728.107143,0.000000,0.000000,725.928571,2.0,2.0,0.0,0.0,1.0,1,27,183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,1316,1,2713.495000,2665.039286,4.0,2.0,2985.423571,2874.767857,400.000000,507.816429,2029.250000,41.0,39.0,4.0,26.0,10.0,1,63,158
965,905,1,573.861071,573.861071,2.0,2.0,634.845357,634.845357,0.000000,0.000000,631.452500,2.0,2.0,0.0,0.0,1.0,0,37,11
966,938,1,448.467143,448.465000,3.0,2.0,470.528571,470.528571,85.714286,221.528571,163.285714,17.0,17.0,2.0,9.0,6.0,0,57,33
967,434,0,173.211786,173.211786,3.0,3.0,174.010714,174.010714,71.428571,9.510714,92.857143,8.0,8.0,4.0,2.0,1.0,0,64,130


In [93]:
#split the data 
from sklearn.model_selection import train_test_split
X = df_train_whole_cl.iloc[:,2:].values
y = df_train_whole_cl.iloc[:,1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [94]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

In [95]:
#random forest 
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
final = RandomForestClassifier(n_estimators = 300,criterion='entropy',random_state=0)
final.fit(X_train, y_train)
y_pred=final.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.7394957983193278

In [96]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

In [97]:
scores = cross_val_score(clf, X_train, y_train, cv=10)
print ('Cross-validated scores:', scores)

Cross-validated scores: [0.76388889 0.70422535 0.73239437 0.67605634 0.70422535 0.77464789
 0.69014085 0.71830986 0.67605634 0.63380282]


In [98]:
y_pred_c = cross_val_predict(clf, X_test, y_test, cv=10)

In [99]:
accuracy = metrics.accuracy_score(y_test, y_pred_c)
accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7394957983193278


In [100]:
# confusion metric
from sklearn.metrics import confusion_matrix
predictions = cross_val_predict(clf, X_train, y_train, cv=10)
cm = confusion_matrix(y_test,y_pred)
cm

array([[155,   7],
       [ 55,  21]])

In [101]:
#logistic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state = 0)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7226890756302521

In [102]:
#decision tree
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.6260504201680672

In [103]:
#naive bayes
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.40336134453781514

In [104]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.6302521008403361

##  Conclusion: Random Forest Classifier for product CL 

In [105]:
# Get the 40% of missing data from df_soc
df_40=pd.merge(df_soc, df_inoutflow, on='Client', how='left')
df_40_1=pd.merge(df_40, df_product, on='Client', how='left')
df_40_1['Sex']=df_40_1['Sex'].replace({'M':'0','F':'1'})
df_40_1 = df_40_1[['Client','VolumeCred','VolumeCred_CA','TransactionsCred','TransactionsCred_CA','VolumeDeb','VolumeDeb_CA','VolumeDebCash_Card','VolumeDebCashless_Card','VolumeDeb_PaymentOrder','TransactionsDeb','TransactionsDeb_CA','TransactionsDebCash_Card','TransactionsDebCashless_Card','TransactionsDeb_PaymentOrder','Age','Tenure']]
#df_train_MF=df_train_MF.dropna()
df_40_2=pd.concat([df_40_1, df_sales])
df_40_f=df_40_2.drop_duplicates(subset='Client', keep=False)
#df_40_f=df_40_f.loc[:,['Client']]
#df_40_f=df_40_f.drop(['Sale_MF','Client','Age','Tenure','Revenue_CC','Revenue_CL','Revenue_MF','Sale_CC','Sale_CL'],axis=1)
#df_40_f=df_40_f.drop(['Sale_MF','Sale_CC','Sale_CL','Revenue_MF','Revenue_CC','Revenue_CL'],axis=1)

df_40_f=df_40_f.loc[:,['VolumeCred','VolumeCred_CA','TransactionsCred','TransactionsCred_CA','VolumeDeb','VolumeDeb_CA','VolumeDebCash_Card','VolumeDebCashless_Card','VolumeDeb_PaymentOrder','TransactionsDeb','TransactionsDeb_CA','TransactionsDebCash_Card','TransactionsDebCashless_Card','TransactionsDeb_PaymentOrder','Sex','Age','Tenure']]
df_40_f=df_40_f.replace({np.NaN:0})
df_40_f



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,VolumeCred,VolumeCred_CA,TransactionsCred,TransactionsCred_CA,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sex,Age,Tenure
0,789.129643,738.230000,4.0,3.0,450.678571,448.892857,178.571429,0.000000,166.571429,8.0,7.0,1.0,0.0,4.0,0.0,21.0,27.0
6,1033.496071,778.370000,8.0,6.0,661.483214,566.126071,89.285714,0.000000,216.892857,13.0,10.0,2.0,0.0,5.0,0.0,37.0,175.0
9,435.682143,435.682143,2.0,2.0,390.056429,390.056429,125.000000,70.842143,190.821429,10.0,10.0,4.0,3.0,2.0,0.0,19.0,70.0
10,132.158929,132.158929,3.0,3.0,3.392857,3.392857,0.000000,0.000000,0.000000,1.0,1.0,0.0,0.0,0.0,0.0,61.0,45.0
13,570.157857,570.157857,2.0,2.0,557.896429,557.896429,71.428571,224.146429,251.178571,17.0,17.0,2.0,10.0,3.0,0.0,41.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,1406.490714,934.384643,7.0,5.0,1404.483571,934.384643,0.000000,0.000000,0.000000,9.0,5.0,0.0,0.0,0.0,0.0,33.0,82.0
1600,178.579643,178.579643,2.0,2.0,109.107143,109.107143,107.142857,0.000000,0.000000,3.0,3.0,2.0,0.0,0.0,0.0,65.0,13.0
1608,811.332857,811.332857,3.0,3.0,577.385714,577.385714,160.714286,65.421429,347.857143,12.0,12.0,2.0,3.0,6.0,0.0,68.0,153.0
1610,469.179643,469.179643,3.0,3.0,465.092857,465.092857,178.571429,11.414286,271.714286,12.0,12.0,1.0,1.0,9.0,0.0,31.0,91.0


In [106]:
sales_CL_prediction=final.predict(df_40_f)
Sales_CL_Prediction=pd.DataFrame(sales_CL_prediction)
Sales_CL_Prediction=Sales_CL_Prediction.rename(columns={0:'Sales_CL_Prediction'})
Sales_CL_Prediction['Sales_CL_Prediction'].value_counts()

0    586
1     60
Name: Sales_CL_Prediction, dtype: int64

## CL product buyers prediction 

In [107]:
Sales_CL_Prediction_final=pd.merge(df_40_f,Sales_CL_Prediction,right_index=True,left_index=True,how='right')
scl=pd.merge(Sales_CL_Prediction_final, df_soc,right_index=True,left_index=True,how='left')
scl=scl[['Client','Sales_CL_Prediction']]
scl=scl[scl.Sales_CL_Prediction!=0]
scl.to_pickle('scl.pickle')

# TABLE REVENUE_CL

In [64]:
# table revenue-cl
df_train_revenue_cl = pd.merge(df_sales, df_inoutflow, on='Client', how='left')
df_train_revenue_cl = df_train_revenue_cl.drop(['Sale_MF','Sale_CC',"Sale_CL",'Revenue_MF','Revenue_CC'],axis=1)
df_train_whole_revenue_cl = pd.merge(df_train_revenue_cl, df_soc, on="Client", how= "left")
df_train_whole_revenue_cl['Sex']=df_train_whole_revenue_cl['Sex'].replace({'M':'0','F':'1'})
df_train_whole_revenue_cl=df_train_whole_revenue_cl.dropna()
df_train_whole_revenue_cl

,Client,Revenue_CL,VolumeCred,VolumeCred_CA,TransactionsCred,TransactionsCred_CA,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sex,Age,Tenure
0,1094,0.000000,0.011786,0.011786,1.0,1.0,1366.313929,1366.313929,53.571429,351.950357,891.327857,31.0,31.0,1.0,21.0,4.0,0,25,8
1,1484,8.722500,1355.088571,1355.088571,5.0,5.0,768.625000,768.625000,107.142857,302.446429,357.785714,23.0,23.0,1.0,16.0,5.0,0,39,0
2,20,0.000000,452.617500,451.550000,4.0,3.0,221.428571,221.428571,107.142857,10.714286,103.571429,7.0,7.0,3.0,1.0,3.0,0,27,181
3,1193,0.000000,1442.837857,1151.652857,23.0,21.0,1661.201071,1207.906429,507.142857,331.150000,75.178571,55.0,36.0,6.0,21.0,2.0,0,48,16
4,142,0.000000,714.285714,714.285714,1.0,1.0,728.107143,728.107143,0.000000,0.000000,725.928571,2.0,2.0,0.0,0.0,1.0,1,27,183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,1316,3.317500,2713.495000,2665.039286,4.0,2.0,2985.423571,2874.767857,400.000000,507.816429,2029.250000,41.0,39.0,4.0,26.0,10.0,1,63,158
965,905,13.166786,573.861071,573.861071,2.0,2.0,634.845357,634.845357,0.000000,0.000000,631.452500,2.0,2.0,0.0,0.0,1.0,0,37,11
966,938,7.812143,448.467143,448.465000,3.0,2.0,470.528571,470.528571,85.714286,221.528571,163.285714,17.0,17.0,2.0,9.0,6.0,0,57,33
967,434,0.000000,173.211786,173.211786,3.0,3.0,174.010714,174.010714,71.428571,9.510714,92.857143,8.0,8.0,4.0,2.0,1.0,0,64,130


In [77]:
#split the data 
from sklearn.model_selection import train_test_split
X_rev = df_train_whole_revenue_cl.iloc[:,2:]
y_rev = df_train_whole_revenue_cl.iloc[:,1]
X_rev_train, X_rev_test, y_rev_train, y_rev_test = train_test_split(X_rev, y_rev, test_size = 0.25, random_state = 0)

In [70]:
from sklearn import linear_model
import statsmodels.api as sm
lm = linear_model.LinearRegression() 
model=lm.fit(X_rev_train, y_rev_train)
y_rev_pred = lm.predict(X_rev_test)

In [79]:
from sklearn.metrics import r2_score
r2_score(y_rev_test, y_rev_pred)

0.06009133362060515

In [80]:
y_pred_train= lm.predict(X_rev_train)
r2_score(y_rev_train, y_pred_train)

0.07017869312405733

In [109]:
Prediction=lm.predict(df_40_f)
Revenue_CL_Prediction=pd.DataFrame(Prediction)
Revenue_CL_Prediction=Revenue_CL_Prediction.rename(columns={0:'Revenue_CL_Prediction'})
Revenue_CL_Prediction=pd.merge(df_40_f,Revenue_CL_Prediction,right_index=True,left_index=True,how='right')
rcl = pd.merge(Revenue_CL_Prediction, df_soc,right_index=True,left_index=True,how='left')
rcl=rcl[['Client','Revenue_CL_Prediction']]
scl = pd.read_pickle('scl.pickle')
Sales_Revenue_CL_Fprediction = pd.merge(scl, rcl, on = 'Client', how = 'left')
Sales_Revenue_CL_Fprediction=Sales_Revenue_CL_Fprediction[Sales_Revenue_CL_Fprediction.Revenue_CL_Prediction>0]
Sales_Revenue_CL_Fprediction=Sales_Revenue_CL_Fprediction.sort_values(by='Revenue_CL_Prediction', ascending=False)
Sales_Revenue_CL_Fprediction.to_pickle('Sales_Revenue_CL_Fprediction.pickle')